In [160]:
-- Nuke database
DROP DATABASE IF EXISTS shipping;
CREATE DATABASE shipping;
USE shipping;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.057

In [161]:
-- This is mysql
-- new table package. make key id, width, lendth, height, weight, price, insurance_id, shipment__id, estimated_distance, estimated_time
--Table Creation
use shipping;
CREATE TABLE package (
    id INT PRIMARY KEY,
    width INT,    -- In centimeters
    length INT,     -- In centimeters
    height INT,   -- In centimeters
    weight INT,  -- In grams
    price DECIMAL(10, 2),  -- In dollars
    type INT,
    insurance_id INT,
    shipment_id INT,
    estimated_distance INT,
    estimated_time_from_dispatch INT,
    time_created TIMESTAMP,
    time_updated TIMESTAMP,
    time_of_dispatch TIMESTAMP,
    time_of_delivery TIMESTAMP,
    delivery_cordinates_x DECIMAL(6,4),
    delivery_cordinates_y DECIMAL(6,4),
    delivery_conformation VARCHAR(255)
);


Create Table shipment_type (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    priority INT,
    description VARCHAR(255),
    days_per500Miles INT,
    cost_per500Miles DECIMAL(10, 2)
);

Create Table insurance (
    id INT PRIMARY KEY,
    type INT,
    price DECIMAL(10, 2),
    value_coverage DECIMAL(10, 2)
);

Create Table insuranceType(
    id INT PRIMARY KEY,
    name VARCHAR(50),
    description VARCHAR(255),
    min_coverage DECIMAL(10, 2),
    max_coverage DECIMAL(10, 2),
    cost_per_dollar DECIMAL(10, 2)
);

Create Table package_status (
    id INT PRIMARY KEY, 
    package_id INT, 
    status INT,
    time_created TIMESTAMP,
    location_x DECIMAL(6,4),
    location_y DECIMAL(6,4),
    location_name VARCHAR(255),
    location_id INT
);
--make id auto increment in package_status
ALTER TABLE package_status MODIFY id INT AUTO_INCREMENT;



Create Table status_info (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    description VARCHAR(255),
    progress_bar_percentage INT,
    tense_name VARCHAR(50)
);
--Based on https://www.drupal.org/project/addressfield
--country => Country (always required, 2 character ISO code)
--name_line => Full name (default name entry)
--first_name => First name
--last_name => Last name
--organisation_name => Company
--administrative_area => State / Province / Region (ISO code when available)
--sub_administrative_area => County / District (unused)
--locality => City / Town
--dependent_locality => Dependent locality (unused)
--postal_code => Postal code / ZIP Code
--thoroughfare => Street address
--premise => Apartment, Suite, Box number, etc.
--sub_premise => Sub premise (unused)
Create Table destination_address (
    id INT PRIMARY KEY,
    country VARCHAR(2),
    name_line VARCHAR(255),
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    organisation_name VARCHAR(255),
    administrative_area VARCHAR(50),
    sub_administrative_area VARCHAR(50),
    locality VARCHAR(50),
    dependent_locality VARCHAR(50),
    postal_code VARCHAR(50),
    thoroughfare VARCHAR(255),
    premise VARCHAR(255),
    sub_premise VARCHAR(255)
);

Create Table Facilities (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    address_id INT,
    type INT,
    arrived_status_code INT,
    departed_status_code INT
);

INSERT INTO Facilities (
    id,
    name,
    address_id,
    type,
    arrived_status_code,
    departed_status_code
) VALUES 
    (0, 'Address' , 0, 0,5,5),
    (1, 'Chicago Distribution Center', 101, 1, 20, 21),
    (2, 'LA Port Facility', 102, 2, 20, 21),
    (3, 'NYC Distribution Hub', 103, 1, 20, 21),
    (4, 'Dallas Sorting Center', 104, 4, 20, 21),
    (5, 'Miami International Hub', 105, 5, 20, 21),
    (6, 'Seattle Port Center', 106, 2, 20, 21),
    (7, 'Denver Local Hub', 107, 3, 22, 23),
    (8, 'Atlanta Distribution Center', 108, 1, 20, 21),
    (9, 'Boston Local Center', 109, 3, 20, 21),
    (10, 'Phoenix Sorting Facility', 110, 4, 20, 21);

Create Table package_assignment (
    id INT PRIMARY KEY,
    package_id INT,
    driver_id INT,
    status INT, -- 0 for not ready, 1 for ready, 2 for dispatched, 3 for completed
    time_assigned TIMESTAMP,
    time_dispatched TIMESTAMP,
    time_completed TIMESTAMP,
    to_address BIT, -- 1 if destination, 0 if other
    from_facility_id INT,
    to_facility_id INT
);

--sample data
INSERT INTO package_assignment (id, package_id, driver_id, time_assigned, time_dispatched, time_completed, to_address, from_facility_id, to_facility_id, status) VALUES (1, 1, 1, '2020-01-01 00:00:00', '2020-01-01 00:00:00', '2020-01-01 00:00:00', 1, 1, 0, 1), (2, 2, 1, '2020-01-01 00:00:00', '2020-01-01 00:00:00', '2020-01-01 00:00:00', 0, 1, 2, 1), (3, 3, 1, '2020-01-01 00:00:00', '2020-01-01 00:00:00', '2020-01-01 00:00:00', 1, 1, 1, 1);
INSERT INTO destination_address (id, country, name_line, first_name, last_name, organisation_name, administrative_area, sub_administrative_area, locality, dependent_locality, postal_code, thoroughfare, premise, sub_premise) VALUES (1, 'US', 'John Doe', 'John', 'Doe', 'Doe Inc.', 'CA', 'Santa Clara', 'Santa Clara', 'Santa Clara', '95050', '123 Main St.', 'Apt 1', '1'), (2, 'US', 'Jane Doe', 'Jane', 'Doe', 'Doe Inc.', 'CA', 'Santa Clara', 'Santa Clara', 'Santa Clara', '95050', '123 Main St.', 'Apt 2', '2'), (3, 'US', 'John Smith', 'John', 'Smith', 'Smith Inc.', 'CA', 'Santa Clara', 'Santa Clara', 'Santa Clara', '95050', '123 Main St.', 'Apt 3', '3');
--all undelivered packages
INSERT INTO package (
    id, 
    width, 
    length, 
    height, 
    weight, 
    price, 
    type, 
    insurance_id, 
    shipment_id, 
    estimated_distance, 
    estimated_time_from_dispatch,
    time_created,
    time_updated,
    time_of_dispatch,
    time_of_delivery,
    delivery_cordinates_x,
    delivery_cordinates_y,
    delivery_conformation
) VALUES 
-- Package 1: Just created, not yet dispatched
(1, 30, 40, 25, 2500, 149.99, 1, 1, 1, 750, 48, 
 '2024-11-30 09:15:00', '2024-11-30 09:15:00', NULL, NULL,
 NULL, NULL, NULL),

-- Package 2: In transit (dispatched but not delivered)
(2, 15, 20, 10, 500, 79.99, 2, 2, 2, 250, 24,
 '2024-11-29 10:00:00', '2024-11-30 09:00:00', '2024-11-29 11:00:00', NULL,
 NULL, NULL, NULL),

-- Package 3: Large item, processed but waiting for dispatch
(3, 100, 120, 80, 15000, 899.99, 1, 3, 1, 1200, 72,
 '2024-11-30 08:30:00', '2024-11-30 10:30:00', NULL, NULL,
 NULL, NULL, NULL);

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

...

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.177

In [162]:
INSERT INTO package_status (package_id, status, time_created, location_x, location_y, location_name, location_id) VALUES
(1, 1, '2024-11-30 09:15:00', 41.8781, -87.6298, 'Chicago Distribution Center', 1),
(1, 9, '2024-11-30 09:30:00', 41.8781, -87.6298, 'Chicago Distribution Center', 1);

-- Package 2: In transit since yesterday, showing movement between facilities
INSERT INTO package_status (package_id, status, time_created, location_x, location_y, location_name, location_id) VALUES
(2, 1, '2024-11-29 10:00:00', 41.8781, -87.6298, 'Chicago Distribution Center', 1),
(2, 10, '2024-11-29 10:30:00', 41.8781, -87.6298, 'Chicago Distribution Center', 1),
(2, 11, '2024-11-29 11:00:00', 41.8781, -87.6298, 'Chicago Distribution Center', 1),
(2, 11, '2024-11-29 18:00:00', 40.7128, -74.0060, 'NYC Distribution Hub', 3),
(2, 11, '2024-11-30 06:00:00', 40.7128, -74.0060, 'NYC Distribution Hub', 3),
(2, 22, '2024-11-30 09:00:00', 40.8000, -73.9500, 'In Transit', 0);

-- Package 3: Large item being processed
INSERT INTO package_status (package_id, status, time_created, location_x, location_y, location_name, location_id) VALUES
(3, 1, '2024-11-30 08:30:00', 41.8781, -87.6298, 'Chicago Distribution Center', 1),
(3, 9, '2024-11-30 09:00:00', 41.8781, -87.6298, 'Chicago Distribution Center', 1);

--Insurance codes
INSERT INTO insuranceType (id, name, description, min_coverage, max_coverage, cost_per_dollar)
VALUES 
(1, 'Basic Parcel', 'Standard coverage for low-value domestic shipments', 100.00, 1000.00, 0.05),

(2, 'Premium Parcel', 'Enhanced coverage for valuable domestic shipments', 1000.01, 5000.00, 0.06),

(3, 'Perishable Goods', 'Coverage for temperature-sensitive and perishable items', 500.00, 10000.00, 0.09);


INSERT INTO status_info (id, name, tense_name, description, progress_bar_percentage) VALUES
-- Initial States
(1, 'Scanned', 'has been scanned', 'Package has been scanned into the system but not yet dispatched', 10),
(2, 'Dispatched', 'is being dispatched', 'Package has been dispatched for delivery', 15),
(3, 'In Transit', 'is in transit', 'Package is in transit between facilities', 25),
(4, 'Processing', 'is being processed', 'Package is being processed at a facility', NULL),
(5, 'Delivered', 'has been delivered', 'Package has been successfully delivered to its destination', 100),
(6, 'Lost', 'has been lost', 'Package has been reported as lost', 0),
(7, 'Damaged', 'has been damaged', 'Package has been reported as damaged', 0),
(8, 'Returned', 'is being returned', 'Package is being returned to sender', 0),
(9, 'Cancelled', 'has been cancelled', 'Delivery has been cancelled', 0),
(10, 'On Hold', 'is on hold', 'Package delivery is temporarily on hold', 0),
(11, 'Out for Delivery', 'is out for delivery', 'Package is out for final delivery', 90),
(12, 'reserved', 'reserved', 'reserved', 0),
(13, 'reserved', 'reserved', 'reserved', 0),
(14, 'reserved', 'reserved', 'reserved', 0),
(15, 'reserved', 'reserved', 'reserved', 0),
(16, 'reserved', 'reserved', 'reserved', 0),
(17, 'reserved', 'reserved', 'reserved', 0),
(18, 'reserved', 'reserved', 'reserved', 0),
(19, 'reserved', 'reserved', 'reserved', 0),


-- Facility Status Codes
(20, 'Arrived at Facility', 'has arrived at facility', 'Package has arrived at a general facility', NULL),
(21, 'Departed from Facility', 'has departed from facility', 'Package has departed from a general facility', NULL),
(22, 'Arrived at Local Facility', 'has arrived at local facility', 'Package has arrived at a local facility', NULL),
(23, 'Departed from Local Facility', 'has departed from local facility', 'Package has departed from local facility', NULL);

-- Insert sample insurance policies aligned with the insurance types
INSERT INTO insurance (id, type, price, value_coverage) VALUES
-- Basic Parcel Insurance Policies
(1, 1, 7.50, 150.00),    -- Basic coverage for Package 1 ($149.99)
(2, 1, 4.00, 80.00),     -- Basic coverage for Package 2 ($79.99)
(3, 2, 54.00, 900.00);

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.007

In [ ]:
-- Constraint Creation
-- id must be a 10 digit number
ALTER TABLE package
ADD CONSTRAINT id_length
CHECK (LENGTH(id) = 10);


--Forign Keys


In [77]:
--functions
INSERT INTO package_status (id, package_id, status, time_created, location_x, location_y) VALUES 
(50, 1, 1, NOW(), 42.4401, -79.3317);

Commands completed successfully

Total execution time: 00:00:00.001

In [78]:
--Sample Data

insert into insurance (id, type, price, value_coverage) values (1, 1, 10.00, 100.00);

INSERT INTO package (
   id, width, length, height, weight, price, type, 
   insurance_id, shipment_id, estimated_distance,
   estimated_time_from_dispatch, time_created, time_updated,
   time_of_dispatch, time_of_delivery, delivery_cordinates_x, delivery_cordinates_y,
   delivery_conformation
) VALUES (
   1234567, 30, 40, 20, 500, 99.99, 1,
   1, 1, 1000, 48,
   NOW(), NOW(),DATE_ADD(NOW(), INTERVAL 1 DAY),
   DATE_ADD(NOW(), INTERVAL 3 DAY), 42.4401, -79.3317, "Left At Front Door"
);



Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.004

In [79]:
select * from package;
select * from package;
INSERT INTO package (
   id, width, length, height, weight, price, type, 
    shipment_id, estimated_distance,
   estimated_time_from_dispatch, time_created, time_updated,
   time_of_dispatch, delivery_cordinates_x, delivery_cordinates_y,
   delivery_conformation
) VALUES (
   1, 30, 40, 20, 500, 99.99, 1,
    1, 1000, 48,
   NOW(), NOW(),
   DATE_ADD(NOW(), INTERVAL 3 DAY), 42.4401, -79.3317, "Left At Front Door"
);
INSERT INTO package (
   id, width, length, height, weight, price, type, 
    shipment_id, estimated_distance,
   estimated_time_from_dispatch, time_created, time_updated,
   time_of_dispatch, time_of_delivery, delivery_cordinates_x, delivery_cordinates_y,
   delivery_conformation
) VALUES (
   1234567, 30, 40, 20, 500, 99.99, 1,
    1, 1000, 48,
   NOW(), NOW(),DATE_ADD(NOW(), INTERVAL 1 DAY),
   DATE_ADD(NOW(), INTERVAL 3 DAY), 42.4401, -79.3317, "Left At Front Door"
);
INSERT INTO package (
   id, width, length, height, weight, price, type, 
    shipment_id, estimated_distance,
   estimated_time_from_dispatch, time_created, time_updated,
   time_of_dispatch, time_of_delivery, delivery_cordinates_x, delivery_cordinates_y,
   delivery_conformation
) VALUES (
   123456, 30, 40, 20, 500, 99.99, 1,
    1, 1000, 48,
   NOW(), NOW(),DATE_ADD(NOW(), INTERVAL 1 DAY),
   DATE_ADD(NOW(), INTERVAL 3 DAY), 42.4401, -79.3317, "Left At Front Door"
);

(2 row(s) affected)

(2 row(s) affected)

Commands completed successfully

Total execution time: 00:00:02.026

id,width,length,height,weight,price,type,insurance_id,shipment_id,estimated_distance,estimated_time_from_dispatch,time_created,time_updated,time_of_dispatch,time_of_delivery,delivery_cordinates_x,delivery_cordinates_y,delivery_conformation
123456,30,40,20,500,99.99,1,NULL,1,1000,48,2024-11-29 23:30:49,2024-11-29 23:30:49,2024-11-30 23:30:49,2024-12-02 23:30:49,42.4401,-79.3317,Left At Front Door
1234567,30,40,20,500,99.99,1,1,1,1000,48,2024-11-29 23:30:58,2024-11-29 23:30:58,2024-11-30 23:30:58,2024-12-02 23:30:58,42.4401,-79.3317,Left At Front Door


id,width,length,height,weight,price,type,insurance_id,shipment_id,estimated_distance,estimated_time_from_dispatch,time_created,time_updated,time_of_dispatch,time_of_delivery,delivery_cordinates_x,delivery_cordinates_y,delivery_conformation
123456,30,40,20,500,99.99,1,NULL,1,1000,48,2024-11-29 23:30:49,2024-11-29 23:30:49,2024-11-30 23:30:49,2024-12-02 23:30:49,42.4401,-79.3317,Left At Front Door
1234567,30,40,20,500,99.99,1,1,1,1000,48,2024-11-29 23:30:58,2024-11-29 23:30:58,2024-11-30 23:30:58,2024-12-02 23:30:58,42.4401,-79.3317,Left At Front Door


: 1062 (23000): Duplicate entry '123456' for key 'package.PRIMARY'